<a href="https://colab.research.google.com/github/aaabhijith13/linkedIN_posts/blob/main/spark_dataframes_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spark with dataframes and SQL

In [37]:
from pyspark.sql import SparkSession, DataFrameReader
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row
from pyspark.sql.functions import col
from pyspark.types import type

In [3]:
spark = SparkSession.builder.getOrCreate() #create session

# Ways to create dataframes:
1. Explicitly mentioning schema and creating datafrmae.
2. Without Schema
3. Changing pandas Dataframe into Spark dataFrame.
4. Reading from path, json, csv etc, as common with Python dataframes.


In [6]:
df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])

In [7]:
pandas_df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [2., 3., 4.],
    'c': ['string1', 'string2', 'string3'],
    'd': [date(2000, 1, 1), date(2000, 2, 1), date(2000, 3, 1)],
    'e': [datetime(2000, 1, 1, 12, 0), datetime(2000, 1, 2, 12, 0), datetime(2000, 1, 3, 12, 0)]
}) #from pandas dataframe
df = spark.createDataFrame(pandas_df)
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [25]:
housing_df = spark.read.csv("sample_data/california_housing_train.csv", header=True, inferSchema=True) #local files, we have header = True because first row is the header in the csv

In [26]:
housing_df.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|
|  -114.57|   33.57|              20.0|     1454.0|         326.0|     624.0|     262.0|        1.925|           65500.0|
|  -114.58|   33.63|    

In [27]:
housing_df.head(2) #shows the two 2 rows with columns and values

[Row(longitude=-114.31, latitude=34.19, housing_median_age=15.0, total_rooms=5612.0, total_bedrooms=1283.0, population=1015.0, households=472.0, median_income=1.4936, median_house_value=66900.0),
 Row(longitude=-114.47, latitude=34.4, housing_median_age=19.0, total_rooms=7650.0, total_bedrooms=1901.0, population=1129.0, households=463.0, median_income=1.82, median_house_value=80100.0)]

In [28]:
housing_df.printSchema() #print schema, you can see that spark infered schema well.

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)



## Common Functions:

In [30]:
housing_df.select("total_rooms").show(3)

+-----------+
|total_rooms|
+-----------+
|     5612.0|
|     7650.0|
|      720.0|
+-----------+
only showing top 3 rows


In [38]:
housing_df.select((col("total_rooms") + col("total_bedrooms")).alias("total_rooms_and_bedrooms")).show() #creating a new row


+------------------------+
|total_rooms_and_bedrooms|
+------------------------+
|                  6895.0|
|                  9551.0|
|                   894.0|
|                  1838.0|
|                  1780.0|
|                  1623.0|
|                  3587.0|
|                   980.0|
|                  5964.0|
|                  1806.0|
|                  4542.0|
|                  2471.0|
|                  1539.0|
|                  2942.0|
|                  1826.0|
|                  3143.0|
|                  2000.0|
|                    77.0|
|                  1774.0|
|                   121.0|
+------------------------+
only showing top 20 rows


In [46]:
modified_df = housing_df.select("*", (col("total_rooms") + col("total_bedrooms")).alias("total_rooms_and_bedrooms"))

Different methods to change columns, you can do select - if you are only planning on selecting few columns.
Ideally for calling all the columns use the function:

```
.withColumn()
```




In [74]:
modified_df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- total_rooms_and_bedrooms: double (nullable = true)



In [50]:
modified_df = housing_df.withColumn(
    "total_rooms_and_bedrooms",
    col("total_rooms") + col("total_bedrooms")
)

Filter and GroupBy functions are some of the common functions, you can do sum, drop, join etc. All functions that are available to Python dataframes and more, to learn about all the functions go here: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.html

In [57]:
modified_df.filter(modified_df["total_rooms_and_bedrooms"] > 1000).show() #filtering when total bedrooms and rooms are above 1000

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|total_rooms_and_bedrooms|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|                  6895.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|                  9551.0|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|                  1838.0|
|  -114.57|   33.57|              20.0|     1454.0|         326.0|     624.0|     262.0|        1.925|           65500

In [59]:
modified_df.groupBy("housing_median_age").count().show() #groupby with count

+------------------+-----+
|housing_median_age|count|
+------------------+-----+
|               8.0|  178|
|               7.0|  151|
|              49.0|  111|
|              29.0|  374|
|              47.0|  175|
|              42.0|  308|
|              44.0|  296|
|              35.0|  692|
|              18.0|  478|
|              39.0|  302|
|               1.0|    2|
|              34.0|  567|
|              37.0|  437|
|              25.0|  461|
|              36.0|  715|
|              41.0|  232|
|               4.0|  161|
|              23.0|  382|
|              50.0|  112|
|              45.0|  235|
+------------------+-----+
only showing top 20 rows


## SQL in Pyspark

You can create a temporary view either in the global view or Session scoped view.

Once established, you can do varied SQL functions like
SELECT to Aggregation, Count etc.

In [64]:
modified_df.createOrReplaceTempView("housing")

In [65]:
sql_df = spark.sql("SELECT * FROM housing")

In [66]:
sql_df

DataFrame[longitude: double, latitude: double, housing_median_age: double, total_rooms: double, total_bedrooms: double, population: double, households: double, median_income: double, median_house_value: double, total_rooms_and_bedrooms: double]

In [67]:
sql_df.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|total_rooms_and_bedrooms|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|                  6895.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|                  9551.0|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|                   894.0|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400

In [73]:
spark.sql("""
SELECT
  CASE
    WHEN total_rooms < 1000 THEN '< 1k'
    WHEN total_rooms < 2000 THEN '1k–2k'
    WHEN total_rooms < 3000 THEN '2k–3k'
    ELSE '3k+'
  END AS room_bucket,
  COUNT(*) AS num_houses,
  ROUND(AVG(median_house_value), 2) AS avg_price
FROM global_temp.housing
GROUP BY room_bucket
ORDER BY avg_price DESC;
""").show()

+-----------+----------+---------+
|room_bucket|num_houses|avg_price|
+-----------+----------+---------+
|        3k+|      4709|233249.71|
|      2k–3k|      4540|215903.73|
|      1k–2k|      5870|190398.32|
|       < 1k|      1881| 174323.0|
+-----------+----------+---------+



In [ ]:
### Let me know if you would like more items here.